# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# TODO: Import the necessary libs
# For example: 
import os

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage
from lib.tooling import tool

In [ ]:
# TODO: Load environment variables
from dotenv import load_dotenv
load_dotenv('config.env')

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [ ]:
import os
import chromadb
from chromadb.utils import embedding_functions
from lib.tooling import tool

@tool
def retrieve_game(query: str, k: int = 5):
    """
    Semantic search: Finds most results in the vector DB

    args:
    - query: a question about game industry.

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    # ---- Chroma client + embedding fn (Vacareum/OpenAI-compatible) ----
    chroma_client = chromadb.PersistentClient(path="chromadb")

    api_key = (os.getenv("OPENAI_API_KEY") or "").strip()
    api_base = (os.getenv("OPENAI_BASE_URL") or "").strip()

    if not api_key:
        raise ValueError("Missing OPENAI_API_KEY in environment.")
    if not api_base:
        raise ValueError("Missing OPENAI_BASE_URL in environment (Vacareum requires this).")

    embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
        api_key=api_key,
        api_base=api_base,
        model_name="text-embedding-3-small",
    )

    # Use get_collection because it should already exist from Part 01
    collection = chroma_client.get_collection(
        name="udaplay",
        embedding_function=embedding_fn,
    )

    # ---- Query ----
    res = collection.query(
        query_texts=[query],
        n_results=int(k),
        include=["metadatas", "documents", "distances"],
    )

    metadatas = (res.get("metadatas") or [[]])[0]
    documents = (res.get("documents") or [[]])[0]
    distances = (res.get("distances") or [[]])[0]

    # ---- Format results into the requested schema ----
    results = []
    for md, doc, dist in zip(metadatas, documents, distances):
        md = md or {}
        results.append(
            {
                "Platform": md.get("Platform"),
                "Name": md.get("Name"),
                "YearOfRelease": md.get("YearOfRelease"),
                "Description": md.get("Description") or doc,
                # helpful extra signal for evaluation/routing
                "distance": dist,
            }
        )

    return results


#### Evaluate Retrieval Tool

In [ ]:
from typing import List, Dict, Any
from pydantic import BaseModel, Field, ValidationError
from lib.tooling import tool
import os
import json

# If Udacity already provides EvaluationReport somewhere, you can replace this
# with: from lib.<wherever> import EvaluationReport
class EvaluationReport(BaseModel):
    useful: bool = Field(..., description="Whether the retrieved docs are sufficient to answer the question.")
    description: str = Field(..., description="Detailed reasoning explaining what is missing or why it's sufficient.")

@tool
def evaluate_retrieval(question: str, retrieved_docs: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Based on the user's question and on the list of retrieved documents,
    it will analyze the usability of the documents to respond to that question.

    args:
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database

    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    # ---- OpenAI/Vacareum compatible client ----
    api_key = (os.getenv("OPENAI_API_KEY") or "").strip()
    base_url = (os.getenv("OPENAI_BASE_URL") or "").strip()  # Vacareum needs this
    if not api_key:
        raise ValueError("Missing OPENAI_API_KEY in environment.")
    if not base_url:
        raise ValueError("Missing OPENAI_BASE_URL in environment (Vacareum requires this).")

    try:
        from openai import OpenAI
    except ImportError as e:
        raise ImportError("openai package not installed. Run: pip install openai") from e

    client = OpenAI(api_key=api_key, base_url=base_url)

    # Keep prompt small + stable
    # Only include the most relevant fields
    compact_docs = []
    for d in (retrieved_docs or [])[:8]:
        compact_docs.append(
            {
                "Platform": d.get("Platform"),
                "Name": d.get("Name"),
                "YearOfRelease": d.get("YearOfRelease"),
                "Description": (d.get("Description") or "")[:300],
                "distance": d.get("distance"),
            }
        )

    system = (
        "You are a strict retrieval evaluator for a game-industry RAG system. "
        "Decide if the retrieved documents are sufficient to answer the user's question accurately. "
        "If the question asks about something not clearly supported (missing game, missing platform, "
        "missing release year, etc.), mark useful=false and explain what’s missing."
    )

    user = (
        "Return ONLY valid JSON with exactly these keys:\n"
        '  {"useful": boolean, "description": string}\n\n'
        f"Question:\n{question}\n\n"
        f"Retrieved documents (may be empty):\n{json.dumps(compact_docs, ensure_ascii=False)}\n"
    )

    # Use a widely supported chat call (works with OpenAI-compatible proxies)
    resp = client.chat.completions.create(
        model="gpt-4o-mini",  # if your course specifies a model, replace here
        messages=[
            {"role": "system", "content": system},
            {"role": "user", "content": user},
        ],
        temperature=0,
    )

    text = (resp.choices[0].message.content or "").strip()

    # Parse JSON robustly
    try:
        data = json.loads(text)
        report = EvaluationReport(**data)
        return report.model_dump()
    except (json.JSONDecodeError, ValidationError):
        # Fallback: attempt to extract JSON block if model wrapped it
        start = text.find("{")
        end = text.rfind("}")
        if start != -1 and end != -1 and end > start:
            try:
                data = json.loads(text[start : end + 1])
                report = EvaluationReport(**data)
                return report.model_dump()
            except Exception:
                pass

        # If parsing fails, return a conservative result
        return EvaluationReport(
            useful=False,
            description=(
                "Evaluator could not produce valid JSON. Raw output was:\n"
                + text[:800]
            ),
        ).model_dump()

#### Game Web Search Tool

In [ ]:
import os
from typing import List, Dict, Any
from lib.tooling import tool

@tool
def game_web_search(question: str, max_results: int = 5) -> List[Dict[str, Any]]:
    """
    Web search: Uses Tavily to search the web for game-industry questions.

    args:
    - question: a question about game industry.

    returns: a list of results, each containing:
    - title: page title
    - url: page url
    - content: short snippet / extracted content (if available)
    - score: relevance score (if provided)
    """
    api_key = (os.getenv("TAVILY_API_KEY") or "").strip()
    if not api_key:
        raise ValueError("Missing TAVILY_API_KEY in environment (.env).")

    try:
        # tavily-python client
        from tavily import TavilyClient
    except ImportError as e:
        raise ImportError("Missing Tavily client. Install with: pip install tavily-python") from e

    client = TavilyClient(api_key=api_key)

    # Tavily returns a dict with a "results" list
    resp = client.search(
        query=question,
        max_results=int(max_results),
        # these are commonly supported; if your version errors, remove them
        include_answer=False,
        include_raw_content=False,
        search_depth="basic",
    )

    results = []
    for r in (resp.get("results") or []):
        results.append(
            {
                "title": r.get("title"),
                "url": r.get("url"),
                "content": r.get("content") or r.get("snippet"),
                "score": r.get("score"),
            }
        )

    return results


### Agent

In [ ]:
import os
from typing import TypedDict, List, Dict, Any
from dotenv import load_dotenv
import contextlib
import io

from lib.llm import LLM
from lib.state_machine import StateMachine, EntryPoint, Termination, Transition, Step
from lib.messages import SystemMessage, UserMessage

load_dotenv(override=True)

OPENAI_API_KEY = (os.getenv("OPENAI_API_KEY") or "").strip()
OPENAI_BASE_URL = (os.getenv("OPENAI_BASE_URL") or "").strip()
TAVILY_API_KEY = (os.getenv("TAVILY_API_KEY") or "").strip()

if not OPENAI_API_KEY:
    raise ValueError("Missing OPENAI_API_KEY")
if not OPENAI_BASE_URL:
    raise ValueError("Missing OPENAI_BASE_URL (Vacareum requires this)")
if not TAVILY_API_KEY:
    raise ValueError("Missing TAVILY_API_KEY")

llm = LLM(model="gpt-4o-mini", api_key=OPENAI_API_KEY)

INSTRUCTIONS = """
You are UdaPlay, an AI research agent for the video game industry.

Goals:
1) Prefer internal knowledge (Vector DB) via retrieve_game (RAG).
2) Evaluate whether retrieved docs are sufficient.
3) If insufficient, search the web using game_web_search.
4) Be precise about platforms and release years.
5) If information is missing, say so clearly.

Answer style:
- Start with a concise direct answer.
- Follow with 2–4 bullet points citing either:
  • retrieved game metadata, or
  • web sources (title + URL).
"""

class UdaPlayState(TypedDict, total=False):
    question: str
    retrieved_docs: List[Dict[str, Any]]
    retrieval_report: Dict[str, Any]
    web_results: List[Dict[str, Any]]
    final_answer: str


def udaplay_workflow(state: UdaPlayState) -> Dict[str, Any]:
    """
    IMPORTANT: Step.run expects this function to return a DICT OF UPDATES,
    not necessarily the full state.
    """
    question = state["question"]

    retrieved = retrieve_game(question)
    report = evaluate_retrieval(question, retrieved)

    updates: Dict[str, Any] = {
        "retrieved_docs": retrieved,
        "retrieval_report": report,
    }

    if not report.get("useful", False):
        web_results = game_web_search(question)
        updates["web_results"] = web_results

        prompt = f"""
User question:
{question}

Web search results:
{web_results}

Evaluator report:
{report}

Answer using the web results.
Include sources (title + URL).
"""
    else:
        prompt = f"""
User question:
{question}

Retrieved documents:
{retrieved}

Evaluator report:
{report}

Answer using ONLY the retrieved documents.
If information is missing, explain what is missing.
"""

    response = llm.invoke([
        SystemMessage(content=INSTRUCTIONS),
        UserMessage(content=prompt),
    ])

    updates["final_answer"] = response
    return updates


# -----------------------------
# Build StateMachine correctly
# -----------------------------
sm = StateMachine(UdaPlayState)

start = EntryPoint()
start.step_id = "START"  # marker step (logic is empty by design)

work = Step("WORK", udaplay_workflow)  # real business logic step

end = Termination()
end.step_id = "END"  # termination marker

sm.add_steps([start, work, end])

# connect uses: connect(source, targets, condition=None)  :contentReference[oaicite:2]{index=2}
sm.connect("START", "WORK")
sm.connect("WORK", "END")
sm.connect("END", [])  # optional; Termination breaks before transition lookup

# -----------------------------
# Run + print final answer
# -----------------------------
with contextlib.redirect_stdout(io.StringIO()):
    run = sm.run({"question": "When Pokémon Gold and Silver was released?"})

final_state = run.get_final_state() or {}
ans = final_state.get("final_answer")
print(ans.content if hasattr(ans, "content") else ans)


In [ ]:
import contextlib, io

# ---------------------------------------
# Invoke your agent (question + answer only)
# ---------------------------------------

questions = [
    "When Pokémon Gold and Silver was released?",
    "Which one was the first 3D platformer Mario game?",
    "Was Mortal Kombat X released for PlayStation 5?"
]

for q in questions:
    # Silence Udacity StateMachine internal prints
    with contextlib.redirect_stdout(io.StringIO()):
        run = sm.run({"question": q})

    final_state = run.get_final_state() or {}
    ans = final_state.get("final_answer")

    print("QUESTION:", q)
    print("ANSWER:", ans.content if hasattr(ans, "content") else ans)
    print()  # blank line between Q&As


### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes